In [2]:
from roboqueue import StationaryTarget, TargetList, Telescope, Conditions, TelescopeSchedulingEnv
from spacerocks.utils import time_handler
import numpy as np
from tabulate import tabulate
import gymnasium as gym
from stable_baselines3 import PPO

In [3]:
telescope = Telescope(obscode='W84', filter='r')
conditions = Conditions(seeing=1, moon=7)

In [4]:
star_positions = [( 24.42813204, -57.23666007), 
                  ( 68.98000195,  16.50976164),
                  ( 78.63446353,  -8.20163919), 
                  ( 79.17206517,  45.99902927),
                  ( 88.79287161,   7.40703634), 
                  ( 95.98787763, -52.69571799),
                  (101.28854105, -16.71314306), 
                  (114.82724194,   5.22750767),
                  (186.64975585, -63.09905586), 
                  (210.95601898, -60.3729784 ),
                  (219.92041034, -60.83514707)]

# convert to radians    
star_positions = [(np.radians(ra), np.radians(dec)) for ra, dec in star_positions]
print(star_positions)

[(0.42635133420991916, -0.9989681710662681), (1.203928152059609, 0.28814969933745876), (1.372430294137903, -0.1431456079261008), (1.381813212820024, 0.8028345134827437), (1.5497279618950985, 0.12927717194787017), (1.6753045066449115, -0.9197137806279083), (1.7678185358639344, -0.2916993747538402), (2.004113442837121, 0.0912372205147572), (3.2576527876260473, -1.1012862796568221), (3.68187710810065, -1.053707252315418), (3.838335252769982, -1.0617736173064813)]


In [5]:
targets = TargetList()

for star in star_positions:
    target = StationaryTarget(ra=star[0], dec=star[1], mag=1, required_snr=1_000_000)
    targets.add_target(target)


In [6]:
epoch = time_handler('12 February 2024').utc.jd[0]

In [8]:
ra, dec, alt, az, exptime = targets.at(epoch=epoch, telescope=telescope, conditions=conditions)
print(tabulate(zip(ra, dec, alt, az, exptime), headers=['RA', 'DEC', 'ALT', 'AZ', 'EXPTIME'], tablefmt='pretty'))
sky_state = np.array([ra, dec, alt, az, exptime]).T
print(f'targets: {targets}')


+---------------------+---------------------+---------------------+---------------------+--------------------+
|         RA          |         DEC         |         ALT         |         AZ          |      EXPTIME       |
+---------------------+---------------------+---------------------+---------------------+--------------------+
| 0.42635133420991916 | -0.9989681710662681 | 0.8393182095028836  | 3.7675534916140676  | 50.46705755912142  |
|  1.203928152059609  | 0.28814969933745876 | 0.7584548029007984  |  6.242530010872558  | 52.070637390610564 |
|  1.372430294137903  | -0.1431456079261008 | 1.1690027201106914  | 0.3548740294287751  | 46.90789848411353  |
|  1.381813212820024  | 0.8028345134827437  | 0.23762084331238295 | 0.10495987958794961 | 113.29560545751062 |
| 1.5497279618950985  | 0.12927717194787017 |  0.851163982662678  | 0.48494994519676426 | 50.26609749585749  |
| 1.6753045066449115  | -0.9197137806279083 | 1.0602194831008114  |  2.584276517797152  | 47.710724959043546 |
|

In [12]:
# Tune the reward weights
reward_config = { 'exptime': 1, 'n_observed_targets': 1 }

# Create training environment
max_duration = 24 * 3600        # [s] fix it later
env = TelescopeSchedulingEnv(telescope, conditions, max_duration, sky_state, targets, reward_config)

# Reset the environment and get the initial observation
observation = env.reset()

# Select an action based on the current observation
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=5000)
model.save("roboqueue")
    
#model = PPO.load("ppo_cartpole")
    
while not done:
    # Take a step in the environment using the selected action
    action, _states = model.predict(obs)
    observation, reward, done, info = env.step(action)
    print(f'observations: {observation}')



[INFO] Observation space dim: Box(-inf, inf, (11, 5), float32), Action space dim: Discrete(12)
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/eshresco/miniconda3/envs/roboqueue/lib/python3.9/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


UnboundLocalError: local variable 'exptime' referenced before assignment

In [ ]:
np.array([exptime, alt]).T